In [56]:
import sys, os
from os.path import join, dirname, abspath
import matplotlib.pyplot as plt
from matplotlib.pyplot import Figure, Axes
import numpy as np
import networkx as nx
import matplotlib.animation as animation
from string import ascii_uppercase
plt.rcParams.update({
    "text.usetex": False,
    "ytick.minor.visible":True,
    "xtick.minor.visible":True,
    'xtick.direction': "in",
    'ytick.direction': "in"
})
outdir = "hw5_out"
os.makedirs(outdir,exist_ok=True)
def out(fname): return join(outdir,fname)
def savefig(plot_name): 
    plt.savefig(out(plot_name),bbox_inches="tight",dpi=250)
import pandas as pd
from numpy.linalg import matrix_power, eig

def arr_to_latex(M):
    return '$$\n' + r'\begin{bmatrix}' + '\n' + (r'\\' + '\n').join('&'.join(str(x) for x in row) for row in M) + '\n' + r'\end{bmatrix}' + '\n' +'$$'

def vec_to_latex(x,round=3):
    return '$$\n' + r'\begin{bmatrix}' + '\n' + (r' \\ ').join(str(np.round(v,round)) for v in x) + '\n' + r'\end{bmatrix}' + '\n' +'$$'
    

# Question 1: Copier I

In [53]:
M = np.array([[0.7,0.5],[0.3,0.5]])
x = [1,0]

In [ ]:
M @ x

In [ ]:
matrix_power(M,2) @ x

In [ ]:
matrix_power(M,7) @ x

In [ ]:
matrix_power(M,31) @ x

In [ ]:
# long-term behavior: eigenvector
vals,vecs=eig(M)
vals,vecs

In [ ]:
vecs[:,0]/np.sum(vecs[:,0])

In [ ]:
vecs[:,1]

# Question 2: Copier II

In [ ]:
M = np.array([[0.699,0.498,0],[0.3,0.5,0],[0.001,0.002,1]])
latex_M = '$$\n' + r'\begin{bmatrix}' + '\n' + (r'\\' + '\n').join('&'.join(str(x) for x in row) for row in M) + '\n' + r'\end{bmatrix}' + '\n' +'$$'
print(latex_M)

In [ ]:
A = M[:2,:2]
A

In [ ]:
F = np.linalg.inv(np.identity(A.shape[0])-A)
F

In [ ]:
print(arr_to_latex(F))

# Question 3: Honeypot

In [ ]:
T = np.array([[0,0,0,0,1],[0,8/13,3/13,1/13,1/13],[1/16,3/16,3/8,1/4,1/8],[0,1/11,4/11,5/11,1/11],[0,1/8,1/2,1/8,1/4]]).T
nu = np.array([0,0,0,0,1])
T

In [ ]:
np.sum(T,axis=0)

In [ ]:
x2=matrix_power(T,2) @ nu
x2

In [ ]:
print(vec_to_latex(x2))

In [ ]:
np.sum(x2)

In [ ]:
x100=matrix_power(T,100) @ nu
x100

In [ ]:
np.sum(x100)

In [ ]:
print(vec_to_latex(x100))

In [ ]:
plt.bar(["80", "135", "139", "445", "No attack"],x100)
plt.xlabel("Port")
plt.ylabel("Chance that Port is Most-Attacked")
plt.title("Honeypot after 100 Weeks")
savefig("honeypot.png")

# Question 4: PageRank
Used to create https://en.m.wikipedia.org/wiki/PageRank#/media/File%3APage_rank_animation.gif

In [64]:
class Page:
    def __init__(self,name) -> None:
        self.inbound = []
        self.outbound = []
        self._outbound = []
        self.name=name
    def set_index(self,idx):
        self.index = idx
    def add_inbound(self,other_page):
        self.inbound.append(other_page)
    def add_outbound(self,*args):
        for p in args:
            p.add_inbound(self)
            self.outbound.append(p)

def build_matrix(pages):
    for i,p in enumerate(pages):
        p.set_index(i)
    M = np.zeros((len(pages),len(pages)))
    for p in pages:
        outbound = list(set([page.index for page in p.outbound if page.index != p.index]))
        p._outbound = outbound
        if outbound:
            # print(outbound)
            M[outbound,p.index] = 1/len(outbound)
        else:
            M[p.index,p.index] = 1
    return M

def PageRank(pages,damping, steps=None, max_iters=1000, convergence_thresh=0.001):
    vals = []
    N = len(pages)
    x = np.empty(N)
    x.fill(1/N)
    M = build_matrix(pages)
    n = 0
    if steps is not None:
        max_iters = steps + 1
    while True:
        if n >= max_iters:
            print(f"PageRank did not converge after {max_iters}")
        if steps is not None:
            if n == steps:
                break
        elif len(vals) > 5:
            changes = np.diff(np.array(vals[-5:]),axis=0)
            total_change = np.sum(np.abs(changes))
            if total_change < convergence_thresh:
                print(f"PageRank converged after {n} iterations")       
                break
        vals.append(np.array(x,copy=True))
        x_prime = np.empty_like(x)
        x_prime.fill((1-damping)/N)
        x_prime += damping * (M @ x)
        x = x_prime
        n+=1
    assert np.allclose(a=sum(x),b=1)
    return x, np.array(vals)

In [65]:
def visualize_network(pages:list,title:str,fig:Figure|None=None,ax:Axes|None=None, colors:dict|None=None, **kwargs) -> tuple[Figure, Axes]:
    build_matrix(pages) # to establish indices lol
    edges = [(p.name,pages[neighbor].name) for p in pages for neighbor in p._outbound]
    G = nx.DiGraph(edges)

    center_index = 2
    center_node = pages[center_index].name

    # Use a spring layout for visualization
    try:
        pos = nx.planar_layout(G)
        raise ValueError  # to force kamada_kawai
    except Exception as e:
        pos = nx.kamada_kawai_layout(G)
        displacement = {node: center_node_position - pos[center_node] for node, center_node_position in pos.items()}
        for node, position in pos.items():
            pos[node] = position + displacement[node]
    if colors:
        c = [colors[name] for name in pos.keys()]
    else:
        c = None
    
    if "node_size" not in kwargs:
        kwargs["node_size"] = 700
    else:
        sizes = dict(zip([p.name for p in pages],kwargs["node_size"]))
        kwargs["node_size"] = [sizes[name] for name in pos.keys()]
        
    # Draw nodes and edges
    if fig is None:
        fig, ax = plt.subplots()
    a = nx.draw(G, pos, with_labels=True, node_color=c, font_size=10,ax=ax, **kwargs)
    ax.set_title(title)
    ax.tick_params(left=False, right=False, labelleft=False,
                    labelbottom=False, bottom=False)
    return fig, ax

In [ ]:
A, B, C, D, E = Page("A"), Page("B"), Page("C"), Page("D"), Page("E")
A.add_outbound(B)
B.add_outbound(E)
E.add_outbound(D)
D.add_outbound(C,E)
C.add_outbound(A,B,E,D)

pages = [A,B,C,D,E]
build_matrix(pages)
visualize_network(pages,"hi")

In [ ]:
R, vals = PageRank(pages,damping=0.85,steps=40)
R

In [ ]:
vals[1,:]-0.03

In [ ]:
vals[1,:]

In [ ]:
np.sum(vals[1,:])

In [ ]:
for i, page in enumerate(pages):
    plt.plot(vals[:,i],label=page.name)
plt.title("PageRank")
plt.xlabel("Iters")
plt.ylabel("PageRank")
_=plt.legend()

In [ ]:
plt.plot(np.sum(np.abs(diffs),axis=1))
plt.ylabel("Total Absolute Change in PageRank")
plt.xlabel("Iteration")
plt.title("Page Rank Convergence")

In [ ]:
for i, page in enumerate(pages):
    plt.plot(diffs[:,i],label=page.name)
plt.title("PageRank")
plt.xlabel("Iters")
plt.ylabel("dPageRank")
_=plt.legend()

In [ ]:
R, vals = PageRank(pages,damping=0.85,steps=None)
R

In [ ]:
# value plot
for i, page in enumerate(pages):
    plt.plot(vals[:,i],label=page.name)
plt.title(f"PageRank ({len(vals)} iters)")
plt.xlabel("Iters")
plt.ylabel("Site PageRank")
_=plt.legend(loc="upper center",ncols=5)
savefig("pagerank_autoconverge.png")
plt.show()
plt.cla()

# difference plot
diffs = np.diff(np.array(vals),axis=0)
for i, page in enumerate(pages):
    plt.plot(diffs[:,i],label=page.name)
plt.title(f"PageRank ({len(vals)} iters)")
plt.xlabel("Iteration")
plt.ylabel("Change in PageRank")
leg=plt.legend(loc="upper center",ncols=5)
savefig("diff_pagerank_autoconverge.png")
plt.show()
plt.cla()

# absolute difference plot
plt.plot(np.sum(np.abs(diffs),axis=1))
plt.title(f"PageRank Convergence ({len(vals)} iters)")
plt.xlabel("Iteration")
plt.ylabel("Total Absolute Change in PageRank")
savefig("abs_diff_pagerank_autoconverge.png")
plt.show()

names = [p.name for p in pages]
colors = [h._color for h in leg.legend_handles]
colors = dict(zip(names,colors))

visualize_network(pages,"",colors=colors,node_size=10000*vals[-1])


In [ ]:
vals[-1]

In [76]:
names = [p.name for p in pages]
colors = [h._color for h in leg.legend_handles]
colors = dict(zip(names,colors))

In [ ]:
np.max(vals)

In [ ]:
fig, ax = plt.subplots()
ymax = np.max(vals)
def anim(frame):
    artists = []
    ax.cla()
    ax.set_ylim(0,ymax)
    b = ax.bar(names,vals[frame,:],color=[colors[n] for n in names])
    ax.set_title(f"PageRank ({len(vals)} iters)")
    ax.set_ylabel(f"Site PageRank")
    return artists

ani = animation.FuncAnimation(fig=fig, func=anim, frames=len(vals), interval=100, repeat=False)
ani.save(filename=out("pagerank.gif"), writer="pillow",dpi=150)

In [ ]:
# for L in ascii_uppercase[:14]:
print(*(ascii_uppercase[:14]),sep=", ",end="")
print(" = ",sep="",end="")
print(*[f"Page(\"{l}\")" for l in ascii_uppercase[:14]], sep=", ")

In [459]:
A, B, C, D, E, F, G, H, I, J, K, L, M, N = Page("A"), Page("B"), Page("C"), Page("D"), Page("E"), Page("F"), Page("G"), Page("H"), Page("I"), Page("J"), Page("K"), Page("L"), Page("M"), Page("N")
A.add_outbound(M)
B.add_outbound(A)
C.add_outbound(A,D)
D.add_outbound(F)
F.add_outbound(D,E)
E.add_outbound(C,D,G)
G.add_outbound(F)
H.add_outbound(G,I)
I.add_outbound(H,J)
J.add_outbound(I,L)
L.add_outbound(K,M)
K.add_outbound(I,J)
M.add_outbound(N)
N.add_outbound(M,L)
pages = [A, B, C, D, E, F, G, H, I, J, K, L, M, N]

In [ ]:
R, vals = PageRank(pages,0.85,steps=None)

In [ ]:
visualize_network(pages,"hi",colors=colors,node_size=5000*vals[-1,:])

In [ ]:
vals[:,3]

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(10,5))

resolution = 5
funcs = []
for p in pages:
    ind = p.index
    f = lambda i,ind=ind: np.interp(i,np.arange(len(vals)),vals[:,ind])
    funcs.append(f)

buffer = 3 * resolution

def update(frame):
    if frame < buffer:  # add a pause at the beginning
        frame = 0
    else:
        frame = (frame-buffer)/resolution
    # v = vals[frame,:]
    v = np.array([f(frame) for f in funcs])
    axes[0].cla()
    axes[1].cla()
    visualize_network(pages,"",colors=colors,node_size=10000*v,fig=fig,ax=axes[0])
    ax = axes[1]
    artists = []
    ax.cla()
    ax.set_ylim(0,ymax)
    b = ax.bar(names,v,color=[colors[n] for n in names])
    ax.set_ylabel(f"Site PageRank")
    
    return [b]

ani = animation.FuncAnimation(fig=fig, func=update, frames=len(vals)*resolution+buffer, interval=300/resolution, repeat=False)
ani.save(filename=out("node.gif"), writer="pillow",dpi=150)

In [ ]:
# for i in range(len(funcs)):
x= np.linspace(0,16,50)
plt.plot(x,funcs[0](x),alpha=0.75)